<a href="https://colab.research.google.com/github/manikcb/all-my-assignments/blob/main/cyberbully3003_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q -U "tensorflow-text==2.8.*"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 498.1/498.1 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 109.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.3/462.3 KB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 109.0 MB/s eta 0:00:00


In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd


In [4]:
df = pd.read_csv("/content/drive/MyDrive/dataset/Untitled folder/mydata3.csv")
df.head(10)

,message,category
0,â??@JuanPee24: â??@_MrHandsomeee: â??@tayyoung...,harass
1,@politicalmoxie no dumb ass @Conservativeind c...,harass
2,"@AndrewS82805912 Andrew, Andrew, Andrew.. Tsk ...",harass
3,"Some dumb fuck really had the nerve to type ""f...",harass
4,All I dumb fuck black got all these different ...,harass
5,Bruh they even said he didnâ??t get it threw y...,harass
6,i'm a dumb nigger that loves to leave my twitt...,harass
7,You know there are people out there who like @...,not_harass
8,"@kebesays no, my blog post. :)",not_harass
9,"Kat and Andre celebrating bad cooking, bad att...",not_harass


In [5]:
df_harass = df[df['category']=='harass']
df_not_harass = df[df['category']=='not_harass']

In [6]:
df_not_harass_downsampled = df_not_harass.sample(df_harass.shape[0],replace=True)
df_balanced = pd.concat([df_not_harass_downsampled, df_harass])
df_balanced['category'].value_counts()


not_harass    710
harass        710
Name: category, dtype: int64

In [7]:
df_balanced['harass']=df_balanced['category'].apply(lambda x: 1 if x=='harass' else 0)
df_balanced.sample(2)

,message,category,harass
1003,BjĂ¶rk too if we're being fair but nobody tell...,harass,1
914,GO TO HELL ! â??@WhitePowerr_: Don't start wit...,harass,1


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_balanced['message'],df_balanced['harass'], stratify=df_balanced['harass'])
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)
model = tf.keras.Model(inputs=[text_input], outputs = [l])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [9]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=METRICS)
model.fit(X_train, y_train, epochs=10)
#y_predicted = model.predict(X_test)
#y_predicted = y_predicted.flatten()
#import numpy as np

#y_predicted = np.where(y_predicted > 0.5, 1, 0)
#y_predicted

Epoch 1/10
34/34 [==============================] - 419s 12s/step - loss: 0.7001 - accuracy: 0.5502 - precision: 0.5483 - recall: 0.5658
Epoch 2/10
34/34 [==============================] - 428s 13s/step - loss: 0.6103 - accuracy: 0.6789 - precision: 0.6939 - recall: 0.6391
Epoch 3/10
34/34 [==============================] - 432s 13s/step - loss: 0.5585 - accuracy: 0.7653 - precision: 0.7640 - recall: 0.7669
Epoch 4/10
34/34 [==============================] - 437s 13s/step - loss: 0.5167 - accuracy: 0.8009 - precision: 0.7749 - recall: 0.8477
Epoch 5/10
34/34 [==============================] - 430s 13s/step - loss: 0.4737 - accuracy: 0.8300 - precision: 0.8293 - recall: 0.8308
Epoch 6/10
34/34 [==============================] - 435s 13s/step - loss: 0.4406 - accuracy: 0.8676 - precision: 0.8641 - recall: 0.8722
Epoch 7/10
34/34 [==============================] - 437s 13s/step - loss: 0.4122 - accuracy: 0.8789 - precision: 0.8752 - recall: 0.8835
Epoch 8/10
34/34 [=======================

In [16]:
# Import pandas library
import pandas as pd

# initialize list elements
x=input("Enter your comment")

data = []
data.append(x)

# Create the pandas DataFrame with column name is provided explicitly
df1 = pd.DataFrame(data, columns=['comment'])
y_predicted = model.predict(df1)
y_predicted = y_predicted.flatten()
import numpy as np



y_predicted = np.where(y_predicted > 0.5, 1, 0)
if y_predicted == 1:
  print("Entered comment is cyberharrased")
else:
  print("entered comment is non cyberharrased")


Enter your commentFuck RTruth. Dumb Nigger
Entered comment is cyberharrased
